<a href="https://colab.research.google.com/github/MarkDaoust/docs/blob/hub_keras/site/en/images/hub_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hub with Keras

In [84]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers

tf.VERSION

'1.12.0-rc1'

In [22]:
import pathlib
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

data_root = pathlib.Path(data_root)
print(data_root)

/root/.keras/datasets/flower_photos


In [0]:
IMAGE_SIZE=224

In [24]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(data_root, target_size=(IMAGE_SIZE, IMAGE_SIZE))

Found 3670 images belonging to 5 classes.


In [25]:
for image_batch,label_batch in image_data:
  print(image_batch.shape)
  print(label_batch.shape)
  break

(32, 224, 224, 3)
(32, 5)


In [0]:
mobilenet_classifier = layers.Lambda(hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/2"))
mobilenet_classifier = tf.keras.Sequential([mobilenet_classifier])

In [0]:
import tensorflow.keras.backend as K
sess = K.get_session()
init = tf.global_variables_initializer()

sess.run(init)

In [28]:
result_batch = mobilenet_classifier.predict(image_batch)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [29]:
import numpy as np
labels_path = tf.keras.utils.get_file('labels.txt','https://raw.githubusercontent.com/googlecodelabs/tensorflow-for-poets-2/master/android/tflite/app/src/main/assets/labels.txt')
labels = np.array(open(labels_path).read().splitlines())

labels[np.argmax(result_batch, axis=-1)]

array(['fig', 'daisy', 'daisy', 'hip', 'chickadee', 'cardoon', 'broom',
       'pot', 'pot', 'spider web', 'picket fence', 'daisy', 'daisy',
       'quill', 'corn', 'daisy', 'corn', 'cardoon', 'cardoon',
       'head cabbage', 'goldfish', 'artichoke', 'tray', 'mushroom',
       'water buffalo', 'breastplate', 'daisy', 'green snake', 'wool',
       'daisy', 'daisy', 'rapeseed'], dtype='<U30')

In [0]:
mobilenet_features_module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2")
mobilenet_features = layers.Lambda(mobilenet_features_module)
mobilenet_features.trainable = False

In [0]:
init = tf.global_variables_initializer()
sess.run(init)

In [0]:
model = tf.keras.Sequential([
  mobilenet_features,
  layers.Dense(image_data.num_classes)
])

In [187]:
result = model.predict(image_batch)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [188]:
result.shape

(32, 5)

In [0]:
model.compile(
  optimizer=tf.train.AdamOptimizer(), 
  loss='categorical_crossentropy')

In [190]:
model.fit_generator(image_data, epochs=1, steps_per_epoch=image_data.samples//image_data.batch_size)

Epoch 1/1
114/114 [==============================] - 81s 709ms/step - loss: 5.5990
